<a href="https://colab.research.google.com/github/karthimd/Mayweather_MD.github.io/blob/main/Res_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import random
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import pickle5 as pickle
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from keras.layers import Dense, Dropout, MaxPooling2D, Flatten, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from keras.layers import Dense, Dropout, MaxPooling2D, Flatten, Conv2D
from tensorflow.keras.applications.resnet50 import ResNet50


In [2]:
!pip install pickle5


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:


def main():
    
    with open("/content/drive/MyDrive/samy/img_train.pkl", "rb") as fh:
        data = pickle.load(fh)
        X_train_ = pd.DataFrame(data)["img_array"] 
    
    with open("/content/drive/MyDrive/samy/img_test.pkl", "rb") as fh:
        data = pickle.load(fh)
        X_test_ = pd.DataFrame(data)["img_array"]
    
    with open("/content/drive/MyDrive/samy/img_y_train.pkl", "rb") as fh:
        data = pickle.load(fh)
        y_train = np.array(pd.DataFrame(data)["label"].values.astype(np.float32)).flatten()
    
    with open("/content/drive/MyDrive/samy/img_y_test.pkl", "rb") as fh:
        data = pickle.load(fh)
        y_test = np.array(pd.DataFrame(data)["label"].values.astype(np.float32)).flatten()

    

    y_test[y_test == 2] = -1
    y_test[y_test == 1] = 2
    y_test[y_test == -1] = 1
    
    y_train[y_train == 2] = -1
    y_train[y_train == 1] = 2
    y_train[y_train == -1] = 1
    

    X_train = []
    X_test = []
    
    for i in range(len(X_train_)):
        X_train.append(X_train_.values[i])
        
    for i in range(len(X_test_)):
        X_test.append(X_test_.values[i])
    
    
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    acc = []
    f1 = []
    precision = []
    recall = []
    
    # Resize images to (224, 224)
    X_train = tf.image.resize(X_train, (224, 224))
    X_test = tf.image.resize(X_test, (224, 224))
    
    # Preprocess images for ResNet50
    X_train = keras.applications.resnet50.preprocess_input(X_train)
    X_test = keras.applications.resnet50.preprocess_input(X_test)

    # Load ResNet50 model
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Add custom classification head
    x = base_model.output
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(128, activation='relu')(x)
    predictions = keras.layers.Dense(3, activation='softmax')(x)
    model = keras.models.Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0.15)
    score = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
    acc.append(score[1])

    test_predictions = model.predict(X_test)
    test_label = to_categorical(y_test, 3)

    true_label = np.argmax(test_label, axis=1)

    predicted_label = np.argmax(test_predictions, axis=1)

    cr = classification_report(true_label, predicted_label, output_dict=True)
    precision.append(cr["macro avg"]["precision"])
    recall.append(cr["macro avg"]["recall"])
    f1.append(cr["macro avg"]["f1-score"])
    
    print("Avg accuracy: " + str(np.array(acc).mean()))
    print("Avg precision: " + str(np.array(precision).mean()))
    print("Avg recall: " + str(np.array(recall).mean()))
    print("Avg f1: " + str(np.array(f1).mean()))
    print("Std accuracy: " + str(np.array(acc).std()))
    print("Std precision: " + str(np.array(precision).std()))
    print("Std recall: " + str(np.array(recall).std()))
    print("Std f1: " + str(np.array(f1).std()))
    print(acc)
    print(precision)
    print(recall)
    print(f1)
    

    
if __name__ == '__main__':
    main()

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/50
77/77 [==============================] - 652s 8s/step - loss: 1.3840 - accuracy: 0.4723 - val_loss: 1.6181 - val_accuracy: 0.2552
Epoch 2/50
77/77 [==============================] - 580s 8s/step - loss: 1.1309 - accuracy: 0.4994 - val_loss: 1.0885 - val_accuracy: 0.5197
Epoch 3/50
77/77 [==============================] - 642s 8s/step - loss: 0.9641 - accuracy: 0.5462 - val_loss: 1.2728 - val_accuracy: 0.5383
Epoch 4/50
77/77 [==============================] - 644s 8s/step - loss: 1.0257 - accuracy: 0.5211 - val_loss: 1.1455 - val_accuracy: 0.4176
Epoch 5/50
77/77 [==============================] - 629s 8s/step - loss: 0.9560 - accuracy: 0.5572 - val_loss: 1.3367 - val_accuracy: 0.3063
Epoch 6/50
77/77 [==============================] - 635s 8s/step - loss: 0.9778 - accuracy: 0.5412 - val_loss: 1.2161 - val_accuracy: 0.4107
Epoch 7/50
77/77 [==============================] - 635s 8s/step - loss: 0.9114 - accurac

In [ ]:
model.save('my_mobilenet_model.h5')

In [3]:
!pip install keras_applications

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install pandas==1.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
